In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import keras
import sklearn

# from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

/Users/Kyelee/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
songs = pd.read_csv('SongCSV.csv', ',')

In [7]:
songs.shape

(10000, 41)

In [9]:
songs.iloc[0:4, :]

,SongNumber,SongID,AlbumID,AlbumName,ArtistFamiliarity,ArtistHotttnesss,ArtistID,ArtistLatitude,ArtistLocation,ArtistLongitude,...,SegmentsStart,SegmentsTimbre,SongHotttnesss,TatumsConfidence,TatumsStart,Tempo,TimeSignature,TimeSignatureConfidence,Title,Year
0,1,b'SOGSMXL12A81C23D88',275907,b'Je Sais Que La Terre Est Plate (Deluxe)',0.557460,0.386152,b'AREJXK41187B9A4ACC',46.71067,b'France',1.71819,...,[ 0. 0.19188 0.60499 0.84939 1.1...,[[ 0. 171.13 9.469 ... 0.973 -10...,0.547953,[0. 0. 0. 0. 0. 0. 0. 0. ...,[4.2570000e-02 2.8357000e-01 5.2458000e-01 7.6...,124.059,4,0.372,b'Je Sais Que La Terre Est Plate',2008
1,2,b'SOMBCOW12AAF3B229F',382807,b'Comme Vous',0.626958,0.434860,b'AR2XRFQ1187FB417FE',NaN,b'',NaN,...,[ 0. 0.51379 0.89138 1.30912 1.6...,[[ 8.00000e-03 1.70986e+02 9.12600e+00 ... ...,0.475638,[0.365 0.321 0.29 0.264 0.238 0.217 0.208 0.1...,[ 0.59161 0.84042 1.08799 1.33555 1.5...,80.084,4,0.533,b'On Efface',2004
2,3,b'SOEYIHF12AB017B5F4',490659,b'Watkins Ale - Music of the English Renaissa...,0.425724,0.000000,b'ARODOO01187FB44F4A',NaN,b'',NaN,...,[ 0. 0.97365 1.34431 1.81351 2.19764 ...,[[ 1.337 145.247 -43.274 ... 16.283 -13...,NaN,[0.307 0.305 0.291 0.284 0.281 0.28 0.276 0.2...,[ 1.11537 1.39701 1.67864 1.95344 2.23097 ...,54.874,4,0.000,b'Howells Delight',0
3,4,b'SODJYEC12A8C13D757',116616,"bDon't Worry Lady""""",0.611495,0.334520,b'ARJGW911187FB586CA',NaN,b'',NaN,...,[0.0000000e+00 7.3060000e-02 3.6862000e-01 6.4...,[[ 14.832 -56.33 69.981 ... 92.933 29...,NaN,[0.731 0.628 0.526 0.359 0.287 0.246 0.21 0.1...,[1.1929000e-01 3.9309000e-01 6.5603000e-01 9.1...,77.150,3,0.369,b'Martha Served',2007


In [29]:
import re

df = songs.values
X = np.column_stack((df[:, 0:5], df[:, 6:]))
y = songs.ArtistHotttnesss

def List(string):
    return(re.findall("\d+\.\d+", string))
    
X[0, 11]

['0.52458',
 '2.4444',
 '4.3658',
 '6.30006',
 '8.22086',
 '10.16615',
 '12.10544',
 '14.03605',
 '15.95314',
 '18.01055',
 '19.98578',
 '21.91698',
 '23.85539',
 '25.7988',
 '27.73742',
 '29.67553',
 '31.60389',
 '33.52611',
 '35.45006',
 '37.39651',
 '39.34086',
 '41.27588',
 '43.20878',
 '45.14084',
 '47.08732',
 '49.03178',
 '50.96292',
 '52.88988',
 '54.82699',
 '56.76412',
 '58.69589',
 '60.63441',
 '62.56993',
 '64.50632',
 '66.44143',
 '68.38683',
 '70.32294',
 '72.25622',
 '74.19204',
 '76.12524',
 '78.06272',
 '79.99603',
 '81.94165',
 '83.87823',
 '85.81078',
 '87.73732',
 '89.656',
 '91.60721',
 '93.54884',
 '95.47982',
 '97.41423',
 '99.34853',
 '101.28531',
 '103.21172',
 '105.1313',
 '107.07421',
 '109.03668',
 '110.97175',
 '112.90656',
 '114.852',
 '116.78805',
 '118.71308',
 '120.64916',
 '122.58551',
 '124.51995',
 '126.44165',
 '128.37186',
 '130.32105',
 '132.26127',
 '134.19859',
 '136.13497',
 '138.07461',
 '140.02416']